In [1]:
import numpy as np
from itertools import combinations_with_replacement

In [2]:
def r2(w_AB, w_Ab, w_aB, n):
    p_AB = w_AB / float(n)
    p_Ab = w_Ab / float(n)
    p_aB = w_aB / float(n)

    p_A = p_AB + p_Ab
    p_B = p_AB + p_aB

    D_ = p_AB - (p_A * p_B)
    denom = p_A * p_B * (1 - p_A) * (1 - p_B)

    if denom == 0 and D_ == 0:
        return np.nan

    return (D_ * D_) / denom

In [3]:
def r(w_AB, w_Ab, w_aB, n):
    p_AB = w_AB / float(n)
    p_Ab = w_Ab / float(n)
    p_aB = w_aB / float(n)

    p_A = p_AB + p_Ab
    p_B = p_AB + p_aB

    D_ = p_AB - (p_A * p_B)
    denom = p_A * p_B * (1 - p_A) * (1 - p_B)

    if denom == 0 and D_ == 0:
        return np.nan

    return D_ / np.sqrt(denom)

In [4]:
def D(w_AB, w_Ab, w_aB, n):
    p_AB = w_AB / float(n)
    p_Ab = w_Ab / float(n)
    p_aB = w_aB / float(n)

    p_A = p_AB + p_Ab
    p_B = p_AB + p_aB

    return p_AB - (p_A * p_B)

In [5]:
def compute_stat_matrix(a_alleles, b_alleles, func, polarized, norm_strategy, do_print=False):
    assert len(a_alleles) == len(b_alleles), 'inputs must be same length'
    assert norm_strategy in {'total', 'hap_weighted', 'af_weighted'}, f'unknown norm strategy: {norm_strategy}'
    a_alleles = np.asarray(a_alleles)
    b_alleles = np.asarray(b_alleles)

    result = np.zeros((2, 2))
    for (l_idx, left), (r_idx, right) in combinations_with_replacement([(0, a_alleles), (1, b_alleles)], 2):
        hap_mat = np.zeros((len(np.unique(right)), len(np.unique(left))))
        for A_i, B_i in zip(left, right):
            hap_mat[A_i, B_i] += 1
        stats, weights = compute_stat(hap_mat, func, polarized, norm_strategy)
        if do_print:
            print('haplotype_matrix', hap_mat, 'stat_result', stats, 'weights', weights, '============', sep='\n')
        result[l_idx, r_idx] = (stats * weights).sum()

    tri_idx = np.tril_indices(len(result), k=-1)
    result[tri_idx] = result.T[tri_idx]
    return result

In [6]:
def compute_stat(hap_mat, func, polarized, norm_strategy):
    hap_mat = np.asarray(hap_mat)

    n_B, n_A = hap_mat.shape
    n = hap_mat.sum()
    a_freq = hap_mat.sum(1) / n
    b_freq = hap_mat.sum(0) / n
    hap_freq = hap_mat / n

    weights = np.zeros(hap_mat.shape)
    stats = np.zeros(hap_mat.shape)
    for A_i in range(1 if polarized else 0, n_A):
        for B_i in range(1 if polarized else 0, n_B):
            w_AB = hap_mat[A_i, B_i]
            w_Ab = hap_mat[A_i,   :].sum() - w_AB 
            w_aB = hap_mat[:  , B_i].sum() - w_AB
            stats[A_i, B_i] = func(w_AB, w_Ab, w_aB, n)
            if norm_strategy == 'hap_weighted':
                weights[A_i, B_i] = hap_freq[A_i, B_i]
            elif norm_strategy == 'af_weighted':
                weights[A_i, B_i] = a_freq[A_i] * b_freq[B_i]
            elif norm_strategy == 'total':
                weights[A_i, B_i] = 1 / ((n_A - (1 if polarized else 0)) * (n_B - (1 if polarized else 0)))
    return stats, weights

Here, we have simplified the representation of the data for testing purposes. Each array represents an allelic state for a set of samples for a given site. Each set has two sites, we can see that sample 0 from site 1 in the CORRELATED dataset has the ancestral state (0).

In [7]:
CORRELATED = (
    [0, 1, 1, 0, 2, 2, 1, 0, 1],
    [1, 2, 2, 1, 0, 0, 2, 1, 2]
)
CORRELATED_SYMMETRIC = (
    [0, 1, 1, 0, 2, 2, 1, 0, 2],
    [1, 2, 2, 1, 0, 0, 2, 1, 0]
)
CORRELATED_BIALLELIC = (
    [0, 0, 0, 0, 1, 1, 1, 1],
    [0, 0, 0, 0, 1, 1, 1, 1]
)
UNCORRELATED = (
    [0, 0, 0, 1, 1, 1, 2, 2, 2],
    [0, 1, 2, 0, 1, 2, 0, 1, 2]
)
UNCORRELATED_BIALLELIC = (
    [0, 0, 0, 0, 1, 1, 1, 1],
    [1, 1, 0, 0, 0, 0, 1, 1]
)
REPULSION_BIALLELIC = (
    [0, 0, 0, 0, 1, 1, 1, 1],
    [1, 1, 1, 1, 0, 0, 0, 0]
)
TEST_CASES = {k: v for k, v in locals().items() if 'CORRELATED' in k or 'ALLELIC' in k}

correlated multi-allelic r2 sums to 1

In this particular case, the alleles are correlated, but there are different numbers of pairs of correlated alleles. The haplotype matrix between sites 0/1 looks like this:

```
[[3. 0. 0.]
 [0. 4. 0.]
 [0. 0. 2.]]
```

Note, the haplotype matrix has A rows and B columns, and the matrix holds counts of A/B haplotypes for each allele.

In [8]:
compute_stat_matrix(*CORRELATED, func=r2, polarized=False, norm_strategy='hap_weighted')

array([[1., 1.],
       [1., 1.]])

We get the same result for the "symmetric" correlated set, which has the same number of pairs of correlated alleles, haplotype matrix between sites 0/1 looks like this:

```
[[3. 0. 0.]
 [0. 3. 0.]
 [0. 0. 3.]]
```

In [9]:
compute_stat_matrix(*CORRELATED_SYMMETRIC, func=r2, polarized=False, norm_strategy='hap_weighted')

array([[1., 1.],
       [1., 1.]])

Now, let's look at unpolarized D, which should always sum to 0 (or very close to it)

In [10]:
for test_case, test_data in TEST_CASES.items():
    print(test_case)
    print(compute_stat_matrix(*test_data, func=D, polarized=False, norm_strategy='total'))

CORRELATED
[[-3.46944695e-18  1.73472348e-18]
 [ 1.73472348e-18  0.00000000e+00]]
CORRELATED_SYMMETRIC
[[0. 0.]
 [0. 0.]]
CORRELATED_BIALLELIC
[[0. 0.]
 [0. 0.]]
UNCORRELATED
[[0. 0.]
 [0. 0.]]
UNCORRELATED_BIALLELIC
[[0. 0.]
 [0. 0.]]
REPULSION_BIALLELIC
[[0. 0.]
 [0. 0.]]


Unpolarized D appears to be incorrect when we perform "hap_weighted" normalization.

In [11]:
for test_case, test_data in TEST_CASES.items():
    print(test_case)
    print(compute_stat_matrix(*test_data, func=D, polarized=False, norm_strategy='hap_weighted'))

CORRELATED
[[0.22222222 0.22222222]
 [0.22222222 0.22222222]]
CORRELATED_SYMMETRIC
[[0.22222222 0.22222222]
 [0.22222222 0.22222222]]
CORRELATED_BIALLELIC
[[0.25 0.25]
 [0.25 0.25]]
UNCORRELATED
[[0.22222222 0.        ]
 [0.         0.22222222]]
UNCORRELATED_BIALLELIC
[[0.25 0.  ]
 [0.   0.25]]
REPULSION_BIALLELIC
[[0.25 0.25]
 [0.25 0.25]]


Now, let's look at "total" normed D for polarized stats, we observe the expected values for biallelic samples. I do think that the correlated multi-allelic samples look a bit fishy (mostly the sign), but I'm not sure what values we would expect.

In [12]:
for test_case, test_data in TEST_CASES.items():
    print(test_case)
    print(compute_stat_matrix(*test_data, func=D, polarized=True, norm_strategy='total'))

CORRELATED
[[ 0.05555556 -0.01851852]
 [-0.01851852  0.04320988]]
CORRELATED_SYMMETRIC
[[ 0.05555556 -0.02777778]
 [-0.02777778  0.05555556]]
CORRELATED_BIALLELIC
[[0.25 0.25]
 [0.25 0.25]]
UNCORRELATED
[[0.05555556 0.        ]
 [0.         0.05555556]]
UNCORRELATED_BIALLELIC
[[0.25 0.  ]
 [0.   0.25]]
REPULSION_BIALLELIC
[[ 0.25 -0.25]
 [-0.25  0.25]]


Let's look at r, which we've also averaged across multi-allelic sites. We see a similar outcome, biallelic behave as we would expect and multiallelic have a similar pattern to D

In [13]:
for test_case, test_data in TEST_CASES.items():
    print(test_case)
    print(compute_stat_matrix(*test_data, func=D, polarized=True, norm_strategy='total'))

CORRELATED
[[ 0.05555556 -0.01851852]
 [-0.01851852  0.04320988]]
CORRELATED_SYMMETRIC
[[ 0.05555556 -0.02777778]
 [-0.02777778  0.05555556]]
CORRELATED_BIALLELIC
[[0.25 0.25]
 [0.25 0.25]]
UNCORRELATED
[[0.05555556 0.        ]
 [0.         0.05555556]]
UNCORRELATED_BIALLELIC
[[0.25 0.  ]
 [0.   0.25]]
REPULSION_BIALLELIC
[[ 0.25 -0.25]
 [-0.25  0.25]]


for a more detailed investigation, one can run with `do_print=True`, which prints out the haplotype comparison matrix, stat result, and weights between sites 0/0, 0/1, 1/1

In [14]:
compute_stat_matrix(*CORRELATED_BIALLELIC, func=D, polarized=True, norm_strategy='total', do_print=True)

haplotype_matrix
[[4. 0.]
 [0. 4.]]
stat_result
[[0.   0.  ]
 [0.   0.25]]
weights
[[0. 0.]
 [0. 1.]]
haplotype_matrix
[[4. 0.]
 [0. 4.]]
stat_result
[[0.   0.  ]
 [0.   0.25]]
weights
[[0. 0.]
 [0. 1.]]
haplotype_matrix
[[4. 0.]
 [0. 4.]]
stat_result
[[0.   0.  ]
 [0.   0.25]]
weights
[[0. 0.]
 [0. 1.]]


array([[0.25, 0.25],
       [0.25, 0.25]])

In [15]:
compute_stat_matrix(*CORRELATED_BIALLELIC, func=D, polarized=False, norm_strategy='total', do_print=True)

haplotype_matrix
[[4. 0.]
 [0. 4.]]
stat_result
[[ 0.25 -0.25]
 [-0.25  0.25]]
weights
[[0.25 0.25]
 [0.25 0.25]]
haplotype_matrix
[[4. 0.]
 [0. 4.]]
stat_result
[[ 0.25 -0.25]
 [-0.25  0.25]]
weights
[[0.25 0.25]
 [0.25 0.25]]
haplotype_matrix
[[4. 0.]
 [0. 4.]]
stat_result
[[ 0.25 -0.25]
 [-0.25  0.25]]
weights
[[0.25 0.25]
 [0.25 0.25]]


array([[0., 0.],
       [0., 0.]])

In [16]:
compute_stat_matrix(*CORRELATED_BIALLELIC, func=r2, polarized=False, norm_strategy='hap_weighted', do_print=True)

haplotype_matrix
[[4. 0.]
 [0. 4.]]
stat_result
[[1. 1.]
 [1. 1.]]
weights
[[0.5 0. ]
 [0.  0.5]]
haplotype_matrix
[[4. 0.]
 [0. 4.]]
stat_result
[[1. 1.]
 [1. 1.]]
weights
[[0.5 0. ]
 [0.  0.5]]
haplotype_matrix
[[4. 0.]
 [0. 4.]]
stat_result
[[1. 1.]
 [1. 1.]]
weights
[[0.5 0. ]
 [0.  0.5]]


array([[1., 1.],
       [1., 1.]])